**Installing the required libraries**

In [1]:
!pip install azure-ai-ml azure-identity

Defaulting to user installation because normal site-packages is not writeable


**Importing the dependencies**

In [2]:
import os
import json
import datetime

from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

**Configure the resource details**

In [3]:
# laoding the configuration file
config_file_path = "config.json"

# Read JSON data into a dictionary
config_data = json.load(open(config_file_path))

subscription_id = config_data["subscription_id"]
resource_group = config_data["resource_group"]  
workspace = config_data["workspace"]
location = config_data["location"]

In [4]:
print(resource_group)
print(workspace)
print(location)

Deploy-rg
Deploy-ws
West Europe


**Resource_Group and Workspace**

**NOTE: Create ResourceGroup and AzureMachineLeanringWorkspace with these names**

In [5]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [6]:
# Define an endpoint name
endpoint_name = "endpt-plant-dis-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="This is an online endpoint for predicting Plant diseases",
    auth_mode="aml_token"
)

In [7]:
print(endpoint_name)

endpt-plant-dis-04080036862745


**Configuring the environment**

**NOTE: Check the quotas to make sure that instance type is available in the chosen region**

vm sku list: https://learn.microsoft.com/en-us/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list?view=azureml-api-2

In [9]:
#Tensorflow Model
model = Model(path=r"C:\Users\venila\Downloads\plant_disease_prediction_model.h5")
env = Environment(
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="onlinescoring", scoring_script="score.py"
    ),
    instance_type="Standard_NC4as_T4_v3",
    instance_count=1,
)

**Creating the endpoint**

In [14]:
%%time
create_endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

CPU times: user 677 ms, sys: 17.3 ms, total: 695 ms
Wall time: 5min 41s


**Deploying the endpoint**

In [15]:
%%time
deploy_endpoint = ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint endpt-plant-dis-01272210986922 exists
Uploading onlinescoring (0.0 MBs): 100%|██| 4700/4700 [00:01<00:00, 4564.31it/s]


Uploading plant_disease_prediction_model.h5 (< 1 MB): 499MB [00:42, 11.8MB/s]   




..............................................................................................................................................CPU times: user 9.24 s, sys: 2.12 s, total: 11.4 s
Wall time: 14min 9s


**You can print create_endpoint and deploy_endpoint and cjheck teh details**

In [16]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
add_traffic = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

**Get details of the endpoint**

In [17]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'blue': 100}
https://endpt-plant-dis-01272210986922.westus.inference.ml.azure.com/score


**Testing the endpoint**

In [18]:
import base64
import json

# Read the image
with open('test_apple_black_rot.JPG', 'rb') as image_file:
    image_data = image_file.read()

# Convert image to base64
image_base64 = base64.b64encode(image_data).decode('utf-8')

json_request = json.dumps({'data': image_base64})

# Save this JSON as a file
json_file_path = 'sample-request.json'  # Specify your desired path for the JSON file
with open(json_file_path, 'w') as json_file:
    json_file.write(json_request)


# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="sample-request.json",
)

In [19]:
print(response)

"{\"Predicted Class Name\": \"Apple___Black_rot\"}"


In [20]:
# Convert the string to a dictionary
prediction = json.loads(response)

# Now dict_data is a Python dictionary
print(prediction)

{"Predicted Class Name": "Apple___Black_rot"}


get AML token: https://learn.microsoft.com/en-us/rest/api/azureml/online-endpoints/get-token?view=rest-azureml-2023-10-01&tabs=HTTP

In [21]:
print(resource_group)
print(workspace)
print(endpoint_name)
print(resource_group)

deploy-rg
deploy-ws
endpt-plant-dis-04080036862745
deploy-rg


Getting the logs

In [ ]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, lines=50
)

Deleting the endpoint

In [ ]:
# deleting the endpoint
#ml_client.online_endpoints.begin_delete(name=endpoint_name)

**NOTE: Clean up the resources**